In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from PIL import Image
import re

def cleanhtml(raw_html):
  cleanr = re.compile('<.*?>')
  cleantext = re.sub(cleanr, '', raw_html)
  return cleantext

In [ ]:
#Combine user data and user comments so password_id can be used to identify botnets

user_data_df = pd.read_csv("/kaggle/input/disqus-social-bot-networks/anonymized_disqus_user_data.csv")
user_comments_df_full = pd.read_csv("/kaggle/input/disqus-social-bot-networks/botnet_comments.csv")
user_meta_data_df = pd.read_csv("/kaggle/input/disqus-social-bot-networks/user_meta_data.csv")
user_comments_df_full = user_comments_df_full.merge(user_data_df, how='left')
user_comments_df_full = user_comments_df_full.merge(user_meta_data_df, how='left')

In [ ]:
#password_1487505 previously identified as an influence campaign
user_comments_df = user_comments_df_full.loc[user_comments_df_full['password_id'] == 'password_1487505']
print(len(user_comments_df))

In [ ]:
import nltk
#Este proceso puede hacerse antes de forma manual, descargar las stopwords de la librería nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words_sp = set(stopwords.words('spanish'))

cleaned_messages = user_comments_df.apply(lambda row : cleanhtml(row['message']), axis = 1)

comments = cleaned_messages.tolist() 
comments = ' '.join(comments).lower()

wordcloud = WordCloud(stopwords = stop_words_sp,
                      collocations=True).generate(comments)

plt.imshow(wordcloud, interpolation='bilInear')
plt.axis('off')
plt.show()

In [ ]:
#List all profile urls
user_comments_df = user_comments_df.sort_values(by="numPosts", ascending=False)
no_dupes = user_comments_df.drop_duplicates(subset='username', keep="last")
profileurls = no_dupes.apply (lambda row: "https://disqus.com/by/" + str(row["username"]), axis=1)

print(profileurls.to_string())